In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns
from scipy.stats import zscore
import math
from sklearn.impute import KNNImputer



In [ ]:
data = pd.read_csv('customer.csv')

# The Unnamed: 0 column resulted from saving the csv file
data.drop('Unnamed: 0', axis=1, inplace=True) 
 
# Replace the null values in the 'is_employed' column with 'False' (empty means no employment)
data.fillna({'is_employed': 'False'}, inplace=True)

data['recent_move_b'] = data['recent_move_b'].apply(lambda x: True if x == 'T' else False)

data['is_employed'] = data['is_employed'].apply(lambda x: True if x == 'True' else False)

data.drop('custid', axis=1, inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72458 entries, 0 to 72457
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   custid          72458 non-null  object 
 1   sex             72458 non-null  object 
 2   is_employed     72458 non-null  bool   
 3   income          72458 non-null  float64
 4   marital_status  72458 non-null  object 
 5   health_ins      72458 non-null  bool   
 6   housing_type    70772 non-null  object 
 7   num_vehicles    70772 non-null  float64
 8   age             72458 non-null  int64  
 9   state_of_res    72458 non-null  object 
 10  code_column     72458 non-null  int64  
 11  gas_usage       70772 non-null  float64
 12  rooms           72458 non-null  int64  
 13  recent_move_b   72458 non-null  bool   
dtypes: bool(3), float64(3), int64(3), object(5)
memory usage: 6.3+ MB


In [24]:
# In the dictionary, we are indicated that the values 1, 2, 3 indicate special cases which we decided to separate
data['gas_payment'] = data['gas_usage'].replace([x for x in data['gas_usage'] if x not in [1, 2, 3]], 0)
data['gas_bill'] = data['gas_usage'].replace([x for x in data['gas_usage'] if x in [1, 2, 3]], 0)

data.drop('gas_usage', axis=1, inplace=True)

In [25]:
# Print the columns with missing values
print(f' --- PRE CLEANING --- \nColumns with missing values: \n{data.isna().sum()}')
old_len = len(data)
print('Length of the dataframe: ', old_len)   

# Drop rows with missing values except age column
data.dropna(inplace=True)


print(f'\n --- POST CLEANING --- \nLength of the dataframe: {len(data)}')
print('Number of rows dropped: ', old_len - len(data))

 --- PRE CLEANING --- 
Columns with missing values: 
custid               0
sex                  0
is_employed          0
income               0
marital_status       0
health_ins           0
housing_type      1686
num_vehicles      1686
age                  0
state_of_res         0
code_column          0
rooms                0
recent_move_b        0
gas_payment          0
gas_bill          1686
dtype: int64
Length of the dataframe:  72458

 --- POST CLEANING --- 
Length of the dataframe: 70772
Number of rows dropped:  1686


In [26]:
data['age'] = data['age'].replace(0, np.nan)

In [33]:
scaling_cols = ['age', 'income', 'gas_payment']


# Dataset with log scaling
data_log = data.copy()
log_cols = np.log1p(data_log[scaling_cols])
data_log[scaling_cols] = log_cols

# Dataset with zscore scaling
data_zs = data.copy()
zs_cols = data_zs[scaling_cols].apply(zscore)
data_zs[scaling_cols] = zs_cols

data_mm = data.copy()

dfs = [data_mm, data_log, data_zs]


for df in dfs:
    numerical_cols = df.select_dtypes(include=[np.number])
    cols = (numerical_cols-numerical_cols.min())/(numerical_cols.max()-numerical_cols.min())
    df[numerical_cols.columns] = cols

c:\Users\castr\anaconda3\envs\IAP2\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)


In [34]:
data_log.head()

,sex,is_employed,income,marital_status,health_ins,housing_type,num_vehicles,age,state_of_res,code_column,rooms,recent_move_b,gas_payment,gas_bill
0,Male,False,0.711953,Never married,True,Homeowner free and clear,0.000000,0.074987,Alabama,0.103726,0.4,False,0.0,0.368421
1,Female,False,0.715735,Divorced/Separated,True,Rented,0.000000,0.778882,Alabama,0.103726,1.0,True,1.0,0.000000
2,Female,False,0.708641,Never married,True,Homeowner with mortgage/loan,0.333333,0.219794,Alabama,0.103726,0.4,False,0.0,0.070175
3,Female,False,0.750436,Widowed,True,Homeowner free and clear,0.166667,0.851887,Alabama,0.103726,0.2,False,0.0,0.210526
4,Male,False,0.752717,Divorced/Separated,True,Rented,0.333333,0.661954,Alabama,0.103726,0.2,False,1.0,0.000000


In [38]:
from sklearn.preprocessing import OneHotEncoder

encoders = []

for i, df in enumerate(dfs):
    categorical_cols = df.select_dtypes(include=[object])
    if categorical_cols.empty:
        continue

    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_array = encoder.fit_transform(categorical_cols)
    encoded_df = pd.DataFrame(
        encoded_array,
        columns=encoder.get_feature_names_out(categorical_cols.columns),
        index=df.index
    )
    
    df.drop(columns=categorical_cols.columns, inplace=True)
    df = pd.concat([df, encoded_df], axis=1)
    encoders.append(encoder)
    dfs[i] = df




In [39]:
dfs[0].head()

,is_employed,income,health_ins,num_vehicles,age,code_column,rooms,recent_move_b,gas_payment,gas_bill,...,state_of_res_South Dakota,state_of_res_Tennessee,state_of_res_Texas,state_of_res_Utah,state_of_res_Vermont,state_of_res_Virginia,state_of_res_Washington,state_of_res_West Virginia,state_of_res_Wisconsin,state_of_res_Wyoming
0,False,0.022866,True,0.000000,0.030303,0.103726,0.4,False,0.0,0.368421,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,False,0.023815,True,0.000000,0.616162,0.103726,1.0,True,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,False,0.022075,True,0.333333,0.101010,0.103726,0.4,False,0.0,0.070175,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,False,0.035343,True,0.166667,0.727273,0.103726,0.2,False,0.0,0.210526,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,0.036316,True,0.333333,0.464646,0.103726,0.2,False,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:

for df in dfs:
    imputer = KNNImputer(n_neighbors=5)
    imputed_data = imputer.fit_transform(df)
    df['age'] = imputed_data[:, 4]

    